In [30]:
import cv2
import numpy as np
import torch
from torch import tensor
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import scipy
from math import sin
from math import cos
from math import radians
from tqdm import tqdm
import math
projects_path = "/home/ibrahim//Environments/TF/Self_Maneuvering_UGV/"

os.chdir(projects_path)
os.getcwd()




'/home/ibrahim/Environments/TF/Self_Maneuvering_UGV'

In [2]:
if (torch.cuda.is_available()):
    device =torch.device("cuda:0")
    print("CUDA")

else:
    device = torch.device("cpu")
    print("CPU")

CUDA


In [57]:
del model
model = torch.nn.Sequential(

        torch.nn.Conv2d(3,  24, stride=(2,2), kernel_size = (5,5), bias=1),
        torch.nn.ReLU(),
        torch.nn.Conv2d(24, 36, stride=(2,2), kernel_size = (5,5), bias=1),
        torch.nn.ReLU(),
        torch.nn.Conv2d(36, 48, stride=(2,2), kernel_size = (5,5), bias=1),
        torch.nn.ReLU(),

        torch.nn.Conv2d(48, 64, stride=(1,1), kernel_size = (3,3), bias=1),
        torch.nn.ReLU(),
        torch.nn.Conv2d(64, 64, stride=(1,1), kernel_size = (3,3), bias=1),
        torch.nn.ReLU(),
        
        torch.nn.Flatten(),

        torch.nn.Linear(27456, 100, bias=1),
        torch.nn.ReLU(),
        torch.nn.Linear(100, 50, bias=1),
        torch.nn.ReLU(),
        torch.nn.Linear(50, 10, bias=1),
        torch.nn.ReLU(),
        torch.nn.Linear(10, 1, bias=1),
)

model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.0001)
loss_function = nn.MSELoss()

print(model[-1])

Linear(in_features=10, out_features=1, bias=True)


In [9]:
tensor_path = projects_path + "saved_tensors"
os.chdir(tensor_path)
os.getcwd()
os.listdir()

dataset_t = torch.load('datasetTensor.pt')
targets_t = torch.load('targetTensor.pt')

dataset_t = dataset_t.float()
targets_t = targets_t.float()

In [23]:
image = dataset_t[0].numpy()
dataset_t[0].shape
cv2.imshow("frame", image)
cv2.waitKey()
cv2.destroyAllWindows()

In [24]:
dataset_t = dataset_t/255.0
print(dataset_t.shape)

dataset_t = dataset_t.permute(0, 3, 1, 2)

torch.Size([8000, 160, 320, 3])


In [58]:
BATCH_SIZE = 100
EPOCHS = 5
epsilon = 0.01
#TODO shuffle dataset

for epoch in range(EPOCHS):
    running_loss = 0
    accuracy = 0
    total = 0
    for i in tqdm(range(0, 8000, BATCH_SIZE)): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
        #print(f"{i}:{i+BATCH_SIZE}")
        batch_X = dataset_t[i:i+BATCH_SIZE].to(device)
        batch_y = targets_t[i:i+BATCH_SIZE].to(device)

        model.zero_grad()

        outputs = model(batch_X)

        with torch.no_grad():
            for j in range(len(batch_y)):
                total +=1
                if(abs(outputs[j] - batch_y[j]) < epsilon):
                    accuracy+=1
                    
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()    # Does the update
        running_loss += loss * i

    print(f"Epoch: {epoch}. acuracy: {accuracy/total}  Loss: {running_loss/8000}")


100%|██████████| 80/80 [00:07<00:00, 11.40it/s]
Epoch: 4. acuracy: 0.0585  Loss: 0.6658434271812439


In [59]:
test1 = dataset_t[1006].unsqueeze(0)
test2 = dataset_t[1008].unsqueeze(0)

print(targets_t[1006])
print(targets_t[1008])

model.to("cpu")
print(test.shape)
print(model(test1))
print(model(test2))


tensor(0.0618)
tensor(0.1862)
torch.Size([1, 3, 160, 320])
tensor([[-0.0440]], grad_fn=<AddmmBackward>)
tensor([[-0.0440]], grad_fn=<AddmmBackward>)


In [60]:
count = 0
for i in range(len(targets_t)):
    if (targets[i] != 0):
        count += 1

print(count)


NameError: name 'targets' is not defined